In [2]:
import keras
import tensorflow as tf
from keras.models import Model
from keras.layers import Dense, Dropout, LSTM, Input, Activation, concatenate
from keras import optimizers
import numpy as np
np.random.seed(4)
#from tensorflow import set_random_seed
tf.random.set_seed(4)
import pandas as pd
from sklearn import preprocessing
import numpy as np

history_points = 50

f='mcd_intraday.csv'

data = pd.read_csv(f)
data = data.drop('date', axis=1)
data = data.drop(0, axis=0)

data = data.values

data_normaliser = preprocessing.MinMaxScaler()
data_normalised = data_normaliser.fit_transform(data)

In [3]:
# using the last {history_points} open close high low volume data points, predict the next open value
ohlcv_histories_normalised = np.array([data_normalised[i:i + history_points].copy() for i in range(len(data_normalised) - history_points)])
next_day_open_values_normalised = np.array([data_normalised[:, 0][i + history_points].copy() for i in range(len(data_normalised) - history_points)])
next_day_open_values_normalised = np.expand_dims(next_day_open_values_normalised, -1)

In [4]:
next_day_open_values = np.array([data[:, 0][i + history_points].copy() for i in range(len(data) - history_points)])
next_day_open_values = np.expand_dims(next_day_open_values, -1)

In [7]:
next_day_open_values

array([[165.51  ],
       [165.5079],
       [165.51  ],
       ...,
       [162.615 ],
       [162.81  ],
       [162.8552]])

In [8]:
data_normalised

array([[1.        , 1.        , 1.        , 0.99507634, 0.14382357],
       [0.97007224, 0.99790804, 0.97750511, 1.        , 0.14131534],
       [0.97420021, 0.9717585 , 0.97137014, 0.97068559, 0.09545623],
       ...,
       [0.27347781, 0.26781306, 0.24744376, 0.24825189, 0.06761065],
       [0.31372549, 0.30128447, 0.26789366, 0.27100832, 0.09488447],
       [0.3230547 , 0.35563366, 0.33120654, 0.31345939, 0.58367477]])

In [11]:
data

array([[1.661350e+02, 1.661500e+02, 1.660800e+02, 1.661100e+02,
        2.334340e+05],
       [1.659900e+02, 1.661400e+02, 1.659700e+02, 1.661338e+02,
        2.293630e+05],
       [1.660100e+02, 1.660150e+02, 1.659400e+02, 1.659921e+02,
        1.549310e+05],
       ...,
       [1.626150e+02, 1.626500e+02, 1.624000e+02, 1.625000e+02,
        1.097360e+05],
       [1.628100e+02, 1.628100e+02, 1.625000e+02, 1.626100e+02,
        1.540030e+05],
       [1.628552e+02, 1.630698e+02, 1.628096e+02, 1.628152e+02,
        9.473380e+05]])

In [30]:
df=pd.DataFrame(data).head(len(next_day_open_values))

In [34]:
df[5]=next_day_open_values
df

,0,1,2,3,4,5
0,166.1350,166.1500,166.0800,166.1100,233434.0,165.5100
1,165.9900,166.1400,165.9700,166.1338,229363.0,165.5079
2,166.0100,166.0150,165.9400,165.9921,154931.0,165.5100
3,166.0900,166.1000,166.0000,166.0150,115897.0,165.5100
4,165.9310,166.1100,165.9310,166.0850,171453.0,165.4476
5,165.9700,165.9900,165.9000,165.9350,95154.0,165.4550
6,165.8550,166.0200,165.8550,165.9600,156964.0,165.4400
7,165.9050,165.9100,165.8200,165.8600,120784.0,165.4900
8,165.7900,165.9100,165.7900,165.9050,140094.0,165.4750
9,165.7450,165.8100,165.7100,165.7800,74596.0,165.4650


In [35]:
df[6]=1-df[5]/df[0]
df

,0,1,2,3,4,5,6
0,166.1350,166.1500,166.0800,166.1100,233434.0,165.5100,0.003762
1,165.9900,166.1400,165.9700,166.1338,229363.0,165.5079,0.002904
2,166.0100,166.0150,165.9400,165.9921,154931.0,165.5100,0.003012
3,166.0900,166.1000,166.0000,166.0150,115897.0,165.5100,0.003492
4,165.9310,166.1100,165.9310,166.0850,171453.0,165.4476,0.002913
5,165.9700,165.9900,165.9000,165.9350,95154.0,165.4550,0.003103
6,165.8550,166.0200,165.8550,165.9600,156964.0,165.4400,0.002502
7,165.9050,165.9100,165.8200,165.8600,120784.0,165.4900,0.002501
8,165.7900,165.9100,165.7900,165.9050,140094.0,165.4750,0.001900
9,165.7450,165.8100,165.7100,165.7800,74596.0,165.4650,0.001689


In [40]:
data=df[[0,1,2,3,4,5]].values

In [44]:
next_day_open_values=df[6].values
next_day_open_values

array([0.003762  , 0.00290439, 0.00301187, ..., 0.00260672, 0.00122692,
       0.00101092])

In [49]:
y_normaliser = preprocessing.MinMaxScaler()
y_normaliser.fit(np.expand_dims(next_day_open_values, -1))

MinMaxScaler(copy=True, feature_range=(0, 1))

In [46]:
def calc_ema(values, time_period):
    # https://www.investopedia.com/ask/answers/122314/what-exponential-moving-average-ema-formula-and-how-ema-calculated.asp
    sma = np.mean(values[:, 3])
    ema_values = [sma]
    k = 2 / (1 + time_period)
    for i in range(len(his) - time_period, len(his)):
        close = his[i][3]
        ema_values.append(close * k + ema_values[-1] * (1 - k))
    return ema_values[-1]

In [47]:
technical_indicators = []
for his in ohlcv_histories_normalised:
    # note since we are using his[3] we are taking the SMA of the closing price
    sma = np.mean(his[:, 3])
    macd = calc_ema(his, 12) - calc_ema(his, 26)
    technical_indicators.append(np.array([sma]))
    # technical_indicators.append(np.array([sma,macd,]))

technical_indicators = np.array(technical_indicators)
technical_indicators

array([[0.90449998],
       [0.90203815],
       [0.89945716],
       ...,
       [0.28934337],
       [0.28724275],
       [0.28542224]])

In [48]:
tech_ind_scaler = preprocessing.MinMaxScaler()
technical_indicators_normalised = tech_ind_scaler.fit_transform(technical_indicators)
technical_indicators_normalised

array([[1.        ],
       [0.99712063],
       [0.99410189],
       ...,
       [0.28051041],
       [0.27805351],
       [0.27592423]])

In [51]:
assert ohlcv_histories_normalised.shape[0] == next_day_open_values_normalised.shape[0] == technical_indicators_normalised.shape[0]
#return ohlcv_histories_normalised, technical_indicators_normalised, next_day_open_values_normalised, next_day_open_values, y_normaliser

In [52]:
ohlcv_histories_normalised

array([[[1.        , 1.        , 1.        , 0.99507634, 0.14382357],
        [0.97007224, 0.99790804, 0.97750511, 1.        , 0.14131534],
        [0.97420021, 0.9717585 , 0.97137014, 0.97068559, 0.09545623],
        ...,
        [0.87719298, 0.87446132, 0.88752556, 0.87808763, 0.01590331],
        [0.87721362, 0.87858249, 0.88836401, 0.87740494, 0.02131039],
        [0.87203302, 0.87866616, 0.88343558, 0.87808763, 0.02804582]],

       [[0.97007224, 0.99790804, 0.97750511, 1.        , 0.14131534],
        [0.97420021, 0.9717585 , 0.97137014, 0.97068559, 0.09545623],
        [0.99071207, 0.98954019, 0.98364008, 0.97542306, 0.07140657],
        ...,
        [0.87721362, 0.87858249, 0.88836401, 0.87740494, 0.02131039],
        [0.87203302, 0.87866616, 0.88343558, 0.87808763, 0.02804582],
        [0.87100103, 0.86820635, 0.8793456 , 0.87198477, 0.01453984]],

       [[0.97420021, 0.9717585 , 0.97137014, 0.97068559, 0.09545623],
        [0.99071207, 0.98954019, 0.98364008, 0.97542306, 0.0

In [53]:
technical_indicators_normalised

array([[1.        ],
       [0.99712063],
       [0.99410189],
       ...,
       [0.28051041],
       [0.27805351],
       [0.27592423]])

In [54]:
next_day_open_values_normalised

array([[0.87100103],
       [0.8705676 ],
       [0.87100103],
       ...,
       [0.27347781],
       [0.31372549],
       [0.3230547 ]])

In [55]:
next_day_open_values

array([0.003762  , 0.00290439, 0.00301187, ..., 0.00260672, 0.00122692,
       0.00101092])

In [56]:
 y_normaliser

MinMaxScaler(copy=True, feature_range=(0, 1))